# Introduction

Suite à la publication de 340 notices d'entités LRM par le groupe Systèmes et Données https://zenodo.org/record/7358799 voici un ensemble de scripts pour visualiser un peu leur contenu

Les données diffusées en iso2709 ont été converties en MarcXML pour faciliter les traitements.

**Plan**
 * [Objectifs](#Objectifs)


 1. **[Construction des données](#1.-Construction-des-donn%C3%A9es)**
   * [Chargement des données initiales](#Chargement-des-donn%C3%A9es-initiales)
     * [Fonctions d'extraction](#Fonctions-d'extraction)
       * [Zones de liens entre entités OEMI](#Zones-de-liens-entre-entit%C3%A9s-OEMI)
       * [Classes et fonctions](#Classes-et-fonctions)
   * [Initialisation des entités](#Initialisation-des-entit%C3%A9s)
   * [Les dictionnaires utiles](#Les-dictionnaires-utiles)
     * [Dictionnaire ID > entité complète](#Dictionnaire-ID-%3E-entit%C3%A9-compl%C3%A8te)
     * [Dictionnaire Manif > Item](#Dictionnaire-Manif-%3E-Items)
     * [Propriétés Oeuvre.toExpressions, Oeuvre.toManifs, Oeuvre.toItems](#Propri%C3%A9t%C3%A9s-Oeuvre.toExpressions,-Oeuvre.toManifs,-Oeuvre.toItems)
     * [Propriétés Expression.toManifs, Expression.toItems](#Propri%C3%A9t%C3%A9s-Expression.toManifs,-Expression.toItems)
     * [Propriétés Manif.toItems, Manif.toOeuvres](#Propri%C3%A9t%C3%A9s-Manif.toOeuvre,-Manif.toItems)
     * [Propriétés Item.toOeuvres, Item.toExpressions](#Propri%C3%A9t%C3%A9s-Item.toOeuvres,-Item.toExpressions)


 2. **[Les entités](#2.-Les-entit%C3%A9s)**
   * [Les oeuvres](#Les-oeuvres)     
     * [Propriété oeuvre.detailed](#Propri%C3%A9t%C3%A9-oeuvre.detailed)
     * [Propriétés de liens aux autres entités : oeuvre.toExpressions, oeuvre.toManifs, oeuvre.toItems](#Propri%C3%A9t%C3%A9s-de-liens-aux-autres-entit%C3%A9s-:-oeuvre.toExpressions,-oeuvre.toManifs,-oeuvre.toItems)
     * [Propriétés oeuvre.resp, oeuvre.respIds](#Propri%C3%A9t%C3%A9s-oeuvre.resp,-oeuvre.respIds)
     * [Liste des titres des oeuvres](#Liste-des-titres-des-oeuvres)
     * [Analyse d'une oeuvre précise](#Analyse-d'une-oeuvre-pr%C3%A9cise)
       * [Expressions et manifestations liées](#Expressions-et-manifestations-li%C3%A9es)
       * [Autres liens](#Autres-liens)
   * [Les expressions](#Les-expressions)
     * [Liste des expressions et liens aux oeuvres](#Liste-des-expressions-et-liens-aux-oeuvres)
   * [Les manifestations](#Les-manifestations)
     * [Liens dans les manifs](#Liens-dans-les-manifestations)
     * [Liste des manifs et liens aux expressions](#Liste-des-manifs-et-liens-aux-expressions)
   * [Les items](#Les-items)
   

 3. **[Relations entre entités](#Relations-entre-entit%C3%A9s)**
   * [Codes fonctions](#Codes-fonctions)
   * [Liens dans les manifs](#Liens-dans-les-manifs)


 4. **[Visualisation en graphe](#3.-Visualisation-en-graphe)**
   * [Générer un graphe Gephi](#G%C3%A9n%C3%A9rer-un-graphe-Gephi)
   * [Avec GraphViz](#Avec-GraphViz)


 * [Coquilles rencontrées](#Coquilles-rencontrées)

In [6]:
# Déclaration des librairies et variables globales

import SRUextraction as sru # import du fichier https://github.com/Lully/bnf-sru/blob/master/SRUextraction.py
from lxml import etree
from string import ascii_lowercase
import re
from collections import defaultdict
import pandas as pd
from pprint import pprint
from IPython.display import FileLink, FileLinks

from unidecode import unidecode

ns = {"marc": "http://www.loc.gov/MARC21/slim"}

# Objectifs

* **Contrôler les données et faire remonter d'éventuels problèmes dans leur contenu ou leur architecture**


* **Fournir des outils pour manipuler les données**, savoir ce qu'elles contiennent. Cela passe par construire un "objet" (Python) intermédiaire pour chaque entité, doté de plusieurs attributs, à savoir :
  * un point d'accès et/ou un label
  * le contenu intégral de la notice
  * la liste des liens vers d'autres entités (avec identifiant et point d'accès)


* **proposer une visualisation pour une entité donnée** :
  * métadonnées simples
  * métadonnées enrichies des entités en liens
  * graphe de liens vers les entités OEMI et autres
  

* **proposer une visualisation pour l'ensemble des données** si cela s'avère pertinent
  
  
* **donner à voir les liens autres que les seuls arbres OEMI** : adaptations ou autres


* **Proposer une simulation d'un moteur de recherche** qui, pour un terme donné, renvoie une liste de résultats. Cela permet de réfléchir à 
  * l'affichage des résultats : quelles informations afficher ? (en imaginant un double affichage simple / détaillé)
  * l'intégration de méatdonnées des entités en lien dans le résultat affiché (pour qu'une recherche puisse s'effectuer à la fois sur des métadonnées de manifestation et d'oeuvre, par exemple)

# 1. Construction des données

## Chargement des données initiales

Nom des 4 fichiers à importer (MarcXML) import des notices en 4 arbres XML

In [ ]:
manifs_filename = "UMB_Manifestations.xml"
oeuvres_expressions_filename = "UMA_Oeuvres_Expressions.xml"
autres_entites_filename = "UMA_Autres_Entites_Liees.xml"
items_filename = "UMH_Items.xml"

Import des fichiers -> conversion en arbres XML

In [ ]:
manifs_file = etree.parse(manifs_filename)
oeuvres_expressions_file = etree.parse(oeuvres_expressions_filename)
autres_entites_file = etree.parse(autres_entites_filename)
items_file = etree.parse(items_filename)

### Fonctions d'extraction

#### Zones de liens entre entités OEMI

#### Classes et fonctions

Import des classes d'objet Record, Item, Manifestation, Expression et Oeuvres

Ce fichier contient aussi le dictionnaire `dic_id2type` : Numéro de notice > Type d'entité

In [ ]:
from Record import *

Import d'un second fichier local avec différents dictionnaires de valeurs
1. Codes fonction (correspondance code Unimarc à 3 chiffres > Libellé). [Source : Abes](https://documentation.abes.fr/sudoc/formats/unmb/DonneesCodees/CodesFonctions.htm#TriCode)
2. Tags de liens entre entités


In [ ]:
from common_dicts import * 

Le fichier Record.py définit **une classe d'objet Record** avec 4 (ou +) sous-classes:
 * Item
 * Manif
 * Expression
 * Oeuvre

Les attributs communs sont 
 * l'identifiant : **.id**
 * le type d'entité : **.type**
 * la notice en XML et en texte : **.xml** et **.txt**
 * un "label" (pas tout à fait un point d'accès en bonne et due forme) : **.label**
 * les statistiques des zones et sous-zones (dictionnaire) : **.stats_zones**
 * les liens aux mentions de responsabilités (dictionnaire) : **.resp**

Chaque entité a en plus un attribut listant (dictionnaire) la ou les entités supérieures auxquelles elle est liée : 
 * items : **.toManifs**
 * manifs : **.toExpressions**
 * expressions : **.toOeuvres**

# Initialisation des entités

On transforme les notices présentes dans les fichiers XML en objets de classes Item, Manif, Expression et Oeuvre telles que définies ci-dessus

In [ ]:
manifs = [Manifestation(manif, "m") for manif in manifs_file.xpath(".//marc:record", namespaces=ns)]
oeuvres_expr = [Record(oe, "oe") for oe in oeuvres_expressions_file.xpath(".//marc:record", namespaces=ns)] 
items = [Item(oe, "i") for oe in items_file.xpath(".//marc:record", namespaces=ns)] 
autres_entites = [Record(r, "p") for r in autres_entites_file.xpath(".//marc:record", namespaces=ns)] 

oeuvres = []
expressions = []
oeuvres_or_expressions  =[]
for entity in oeuvres_expr:
    if entity.type == "o":
        oeuvre = Oeuvre(entity.xml, entity.init_type)
        oeuvres.append(oeuvre)
        
    elif entity.type == "e":
        expression = Expression(entity.xml, entity.init_type)
        expressions.append(expression)
    else:
        oeuvres_or_expressions.append(entity)

On peut donc désormais appeler manipuler les informations présentes dans ces entités : on dispose de 4 listes (items, manifs, oeuvres, expressions) 

par exemple, pour la liste des manifestations avec leur identifiant et leur label, écrire :

```for manif in manifs:
    print(manif.id, manif.label)```

Vous pouvez aussi vous demander à quoi ressemble une notice d'item en Unimarc, en affichant un des items de la liste `items`:

`print(items[0].txt)`

ou le 2e : 

`print(item[1]).txt`

**Distinction entre oeuvres et expressions dans le fichier oeuvres_expressions.xml**

Label pos.9 permet de distinguer oeuvres et expressions : valeurs h / t (auteur-titre, ou titre seul) 

Zone 154 $a pos1 : a = oeuvre ; b = expression ; x = ne s'applique pas

Point d'accès
* 231 pour les oeuvres
* 232 pour les expressions

## Les dictionnaires utiles et les attributs à enrichir

Ensemble de dictionnaires permettant de naviguer dans les entités à partir d'un élément d'information

En outre, les liens indirects (Oeuvre > Manifestation, Item > Expression) doivent être générés lors d'un second passage sur les lots d'entités, pour que chaque objet dispose de toutes ces informations comme des propriétés qui lui soient rattachées

### Dictionnaire ID > entité complète

A partir de l'identifiant (numéro de notice) on récupère l'entité de classe Item, Manif, Expression ou Oeuvre (au sens des classes définies dans le code ci-dessus

In [ ]:
# dict_entities importé du fichier Record.py
for entity in items + manifs + expressions + oeuvres + autres_entites:
    dict_entities[entity.id] = entity

### Dictionnaire Manif > Items

Chaque notice contient des liens vers l'entité "supérieure" dans l'arbre FRBR. Mais les liens inverses peuvent être utiles.

On en profite pour récupérer les points d'accès des manifs dans les liens Item > Manif (au sein de l'item)

In [ ]:
dict_manif2item = defaultdict(list)
for item in items:
    for manif in item.toManifs:
        try:
            manif_accesspoint = dict_entities[manif].label
        except KeyError:
            print("Manifestation", manif, f"mentionnée dans l'item {item.id} mais absente du fichier des manifestations")
        item.toManifs[manif] = manif_accesspoint
        dict_manif2item[manif].append(item.id)

### Propriétés Oeuvre.toExpressions, Oeuvre.toManifs, Oeuvre.toItems

In [ ]:
for expr in expressions:
    for oeuvre in expr.toOeuvres:
        dict_entities[oeuvre].toExpressions[expr.id] = expr.label
for manif in manifs:
    for expr in manif.toExpressions:
        if expr:
            for oeuvre in dict_entities[expr].toOeuvres:
                dict_entities[oeuvre].toManifs[manif.id] = manif.label
for item in items:
    for manif in item.toManifs:
        for expr in dict_entities[manif].toExpressions:
            if expr:
                for oeuvre in dict_entities[expr].toOeuvres:
                    dict_entities[oeuvre].toItems[item.id] = item.label
# régénération de la liste des oeuvres
oeuvres = [dict_entities[o] for o in dict_entities if dict_entities[o].type == "o"]

### Propriétés Expression.toManifs, Expression.toItems

In [ ]:
for manif in manifs:
    for expr in manif.toExpressions:
        if expr:
            dict_entities[expr].toManifs[manif.id] = manif.label
for expr in dict_entities:
    if dict_entities[expr].type == "e":
        for manif in dict_entities[expr].toManifs:
            items_list = dict_manif2item[manif]
            for item in items_list:
                dict_entities[expr].toItems[item] = dict_entities[item].label
# régénération de la liste des expressions
expressions = [dict_entities[e] for e in dict_entities if dict_entities[e].type == "e"]

### Propriétés Manif.toOeuvre, Manif.toItems

In [ ]:
for manif in manifs:
    for expr in manif.toExpressions:
        if expr:
            oeuvres_list = dict_entities[expr].toOeuvres
            for oeuvre in oeuvres_list:
                dict_entities[manif.id].toOeuvres[oeuvre] = dict_entities[oeuvre].label
    for item in dict_manif2item[manif.id]:
        dict_entities[manif.id].toItems[item] = dict_entities[item].label
# régénération de la liste des expressions
manifs = [dict_entities[m] for m in dict_entities if dict_entities[m].type == "m"]

### Propriétés Item.toOeuvres, Item.toExpressions

In [ ]:
for item in items:
    for manif in item.toManifs:
        manif_entity = dict_entities[manif]
        for expr in manif_entity.toExpressions:
            if expr:
                dict_entities[item.id].toExpressions[expr] = dict_entities[expr].label
        for oeuvre in manif_entity.toOeuvres:
            dict_entities[item.id].toOeuvres[oeuvre] = dict_entities[oeuvre].label

# 2. Les entités

## Les oeuvres

In [ ]:
print("Nombre d'oeuvres : ", len(oeuvres))

Exemple d'une oeuvre, avec son contenu et ses enrichissements

In [ ]:
print(dict_entities["UMLRM0163"])

#### Propriété oeuvre.detailed

In [ ]:
print(dict_entities["UMLRM0163"].detailed)

#### Propriétés de liens aux autres entités : oeuvre.toExpressions, oeuvre.toManifs, oeuvre.toItems

In [ ]:
print("Oeuvre.toExpressions : ", dict_entities["UMLRM0163"].toExpressions)
print("Oeuvre.toManifs      : ", dict_entities["UMLRM0163"].toManifs)
print("Oeuvre.toItems       : ", dict_entities["UMLRM0163"].toItems)

#### Propriétés oeuvre.resp, oeuvre.respIds

Liens aux mentions de responsabilités

In [ ]:
print("Oeuvre.resp    : ", dict_entities["UMLRM0163"].resp)
print("\n")
print("Oeuvre.respIds : ", dict_entities["UMLRM0163"].respIds)

### Liste des titres des oeuvres

In [ ]:
liste_titres_oeuvres = []

for oeuvre in oeuvres:
    liste_titres_oeuvres.append({"id": oeuvre.id,"Label": oeuvre.label, 
                                 "Nb Expressions liées": len(oeuvre.toExpressions),
                                 "Nb Manifs liées": len(oeuvre.toManifs),
                                 "Nb Items liés": len(oeuvre.toItems)})
liste_titres_oeuvres = pd.DataFrame(liste_titres_oeuvres)
# liste_titres_oeuvres.to_excel("Liste des oeuvres.xlsx", encoding="utf-8")
liste_titres_oeuvres

## Oeuvres et expressions liées

Après un peu de mise en forme du [tableau ci-contre, au format Excel](Liste%20des%20oeuvres_mef.xlsx), on voit ressortir l'oeuvre la plus reprise et éditée, avec 10 expressions et 20 manifestations liées : Dracula, de Bram Stocker


## Analyse d'une oeuvre précise

Prenons le temps de regarder cette notice d'oeuvre UMLRM0004 : Bram Stoker, Dracula, 1890

In [ ]:
oeuvre_zoom = "UMLRM0004"

In [ ]:
print(dict_entities[oeuvre_zoom])

### Expressions et manifestations liées

In [ ]:
for expr in dict_entities[oeuvre_zoom].toExpressions:
    print("\n", dict_entities[expr].detailed)
    for manif in dict_entities[expr].toManifs:
        print(" "*5, "|---", manif, dict_entities[manif].label)
#    print(expr, dict_entities["UMLRM0004"].toExpressions[expr])

### Autres liens

In [ ]:
i = 1
for oeuvre in oeuvres:
    f541_3 = sru.record2fieldvalue(oeuvre.xml, "541$3")
    if f541_3 == oeuvre_zoom:
        lien = [sru.record2fieldvalue(oeuvre.xml, "541$p") + " " + sru.record2fieldvalue(oeuvre.xml, "541$t"), sru.record2fieldvalue(oeuvre.xml, "541$a")]
        lien = [el.strip() for el in lien if el.strip()]
        print("")
        print(i, "-", oeuvre.label, f"({oeuvre.id})")
        print(", ".join(lien))
        i += 1

**Items liés**

In [ ]:
for item in dict_entities["UMLRM0004"].toItems:
    print(item, dict_entities["UMLRM0004"].toItems[item])

## Les expressions

Les expressions sont présentes dans le fichier oeuvres_expressions, et distinguables les une des autres par la zone **154$a pos.1**, selon la valeur trouvée : 
* a : oeuvre
* b : expression
* x : autres

### Liens Expressions > mentions de responsabilités

In [ ]:
stats_exp_resp = defaultdict(int)

for expr in expressions:
    stats_exp_resp[len(expr.resp)] += 1

for key in stats_exp_resp:
    print("Nombre d'expressions contenant", key, "mentions de responsabilités : ", stats_exp_resp[key])

### Fichier contenant les expressions

In [ ]:
def line2report(line, file):
    file.write(line)
    file.write("\n")

with open("expressions.txt", "w", encoding="utf-8") as f:
    for expr in expressions:
        line2report("\n"*2, f)
        line2report(f"{expr.id} : {expr.label}", f)
        line2report(expr.txt, f)

### Les liens dans les expressions

Expression UMLRM0056

## Les manifestations

Besoins : 
* pour une manifestation, récupérer les métas principales :
  * titre, point d'accès, auteur
  * liens aux entités Expressions (et oeuvre ?)
* pour une manif, récupérer la liste des expressions et la liste des oeuvres (lien direct)
  * créer des attributs à la manifs, en fournissant l'expression source pour chaque oeuvre identifié

## Les items

In [ ]:
print("Nombre d'items : ", len(items))

Liste des numéros de notices d'item

In [ ]:
print([item.id for item in items])

**Visualiser un item par son numéro de notice (liste ci-dessus) :**

In [ ]:
# Fonction désactivée temporairement
# pour facilités de développement -- mais elle fonctionne

"""itemid = input("Numéro d'item (exemple UMLRM1050): ")
if itemid == "":
    itemid = "UMLRM1050"
print(dict_entities[itemid])"""

Chaque item n'est relié qu'à 1 manifestation

In [ ]:
stats_manifs_items = defaultdict(int)
for manif in dict_manif2item:
    stats_manifs_items[len(dict_manif2item[manif])] += 1
for nb_items in stats_manifs_items:
    print(stats_manifs_items[nb_items], "manifestations reliées à", nb_items, "items")
for manif in dict_manif2item:
    if len(dict_manif2item[manif]) > 1:
        print(manif, "liée à", len(dict_manif2item[manif]), "items")

# Relations entre entités

Note (non exploitable informatiquement) 370


## Codes fonctions

In [ ]:
liste_roles = set()
for e in manifs + expressions + oeuvres:
    for r in e.resp:
        for role in e.resp[r]:
            liste_roles.add(f"Entité {e.type} - {role}")
for el in sorted(liste_roles):
    print(el)

## Liens dans les manifs

### Liens manif > expression

In [ ]:
print("*** Liens aux Expressions ***")
for lien in tags_manif2expressions:
    print(lien, tags_manif2expressions[lien])

* Chaque manifestation a un attribut **.toExpression** qui est un dictionnaire :
  * clé : numéro de notice d'expression
  * valeur : point d'accès (sans les dollars)

**Liste des manifestations et liens aux expressions**

In [ ]:
for el in manifs:
    print("\n", el.id, el.label)
    for expression in el.toExpressions:
        print(f"    ∟{'-'*5}Expr{'-'*2}", expression, el.toExpressions[expression])

### Mentions de responsabilités

In [ ]:
print("*** Liens aux mentions de responsabilités ***")
for lien in tags_resp["m"]:
    print(lien, tags_resp["m"][lien])

* Chaque manifestation a un attribut **.resp** qui est un dictionnaire:
  * clé : label de l'entité en lien
  * valeur : liste des fonctions

## Les expressions

### Liste des expressions et liens aux oeuvres

Chaque instance de l'entité Expression a un attribut ".toOeuvres" qui liste sous forme de dictionnaire les liens aux oeuvres (clé : numéro de notice d'oeuvre ; valeur : point d'accès de l'oeuvre)

In [ ]:
for el in expressions:
    print("\n", el.id, el.label)
    for oeuvre in el.toOeuvres:
        print(f"    ∟{'-'*5}", oeuvre, el.toOeuvres[oeuvre])

In [ ]:
# Mentions de responsabilités dans les expressions:

for exp in expressions:
    for field in [str(el) for el in range(500, 599)]:
        for field_occ in exp.xml.xpath(f"*[@tag='{field}']"):
            value = sru.field2value(field_occ)
            if "$4" in value:
                print(exp.type, field, value)

# Visualisation en graphe

## Générer un graphe Gephi

Objectif : générer 2 fichiers TSV (format en colonnes, tabulé) afin d'alimenter Gephi et de calculer un graphe

Un fichier listera les noeuds : 
 * identifiant (n° de notice)
 * label (point d'accès)
 * type (manifestation/expression/oeuvre) (à voir la pertinence d'y mettre aussi les items)

Un second fichier listera les liens entre ces noeuds
 

### Fichier - liste des noeuds

In [ ]:
liste_noeuds = [["Identifiant", "Label", "Type"]]

def add_in_nodes_list(corpus, liste):
    for entity in corpus:
        liste.append([entity.id, entity.label, entity.type])

add_in_nodes_list(manifs, liste_noeuds)
add_in_nodes_list(expressions, liste_noeuds)
add_in_nodes_list(oeuvres, liste_noeuds)
add_in_nodes_list(items, liste_noeuds)
add_in_nodes_list(autres_entites, liste_noeuds)

### Fichier - liste des liens

In [ ]:
liste_liens = []
for item in items:
    for manif in item.toManifs:
        liste_liens.append([item.id, manif])
for manif in manifs:
    for expr in manif.toExpressions:
        liste_liens.append([manif.id, expr])
for expr in expressions:
    for oeuvre in expr.toOeuvres:
        liste_liens.append([expr.id, oeuvre])
for entity in dict_entities:
    for resp in dict_entities[entity].respIds:
        liste_liens.append([entity, resp])
for oeuvre in oeuvres:
    for subject in oeuvre.subjects:
        liste_liens.append([oeuvre.id, subject])
    for gf in oeuvre.genreforme:
        liste_liens.append([oeuvre.id, gf])

In [ ]:
print("Exemple de relations exprimées par des points d'accès seul : ")
for lien in liste_liens:
    for recordid in lien:
        if re.fullmatch(r"UMLRM\d\d\d\d", recordid) is None:
            print(dic_id2type[lien[0]], lien)

## Génération des fichiers

In [ ]:
with open("liste_noeuds.txt", "w", encoding="utf-8") as file:
    for node in liste_noeuds:
        file.write("\t".join(node) + "\n")
with open("liste_liens.txt", "w", encoding="utf-8") as file:
    for link in liste_liens:
        file.write("\t".join(link) + "\n")

### Graphe GEXF

On peut aussi utiliser Gephi pour générer un fichier \*.gexf avec une librairie JavaScript qui va générer un graphe interactif dans le navigateur

On fournit à Gephi les noeuds et liens, pour avoir la projection dans le plan.

## Avec GraphViz

Cf. test sur [../graphviz/Test Graphviz.ipynb](graphviz/Test Graphviz.ipynb)

In [ ]:
import graphviz

### Fonctions de génération d'un graphe à partir d'une oeuvre

In [ ]:

def generate_graph_from_oeuvre(oeuvre, size):
    # A partir d'une manifestation, renvoie un graphe GraphViz
    dot = graphviz.Digraph(oeuvre.id, comment=f'Notice {oeuvre.id}')  
    dot.attr(rankdir='LR')
    dot.attr(size=size)
    # Add nodes and edges to the graph object using its node() and edge() or edges() methods:
    dot.node(oeuvre.id, oeuvre.label)
    liste_edges = []
    for expr in oeuvre.toExpressions:
        if expr:
            dot.node(expr, dict_entities[expr].label)
            dot.edge(oeuvre.id, expr, label="Expression de")
        for manif in dict_entities[expr].toManifs:
            if manif:
                dot.node(manif, dict_entities[manif].label)
                dot.edge(expr, manif, label="Manifestation de")
    return dot
    # dot.edges(liste_edges)
    
def oeuvreid2graph(recordid, size="10,10"):
    return generate_graph_from_oeuvre(dict_entities[recordid], size)

In [ ]:
graphe_oeuvre = oeuvreid2graph("UMLRM0129", "15,15")
graphe_oeuvre

In [ ]:
graphe_oeuvre = oeuvreid2graph("UMLRM0129")
graphe_oeuvre

### Fonctions de génération d'un graphe à partir d'une manifestation

In [ ]:

def generate_graph_from_manif(manif):
    # A partir d'une manifestation, renvoie un graphe GraphViz
    dot = graphviz.Digraph(manif.id, comment=f'Notice {manif.id}')  
    # Add nodes and edges to the graph object using its node() and edge() or edges() methods:
    dot.node(manif.id, manif.label)
    liste_edges = []
    for expr in manif.toExpressions:
        if expr:
            dot.node(expr, dict_entities[expr].label)
            dot.edge(manif.id, expr, label="manif de")
        for oeuvre in dict_entities[expr].toOeuvres:
            if oeuvre:
                dot.node(oeuvre, dict_entities[oeuvre].label)
                dot.edge(expr, oeuvre, label="expression de")
    return dot
    # dot.edges(liste_edges)
    
def manifid2graph(recordid):
    return generate_graph_from_manif(dict_entities[recordid])

[Liste des notices de manifestations ci-dessus](#Liste-des-manifs-et-liens-aux-expressions)

In [ ]:
# Fonction désactivée temporairement
# pour facilités de développement -- mais elle fonctionne

# manifid = input("Numéro de notice de manifestation (exemple : UMLRM0077, avec 2 expressions liées) : ")
manifid = ""

In [ ]:
if manifid == "":
    manifid = "UMLRM0077"
graphe_manif = manifid2graph(manifid)

In [ ]:
graphe_manif

### Générer un graphe complet

In [ ]:
def generate_global_graph(dict_entities, list_links, size):
    # A partir d'une manifestation, renvoie un graphe GraphViz
    dot = graphviz.Digraph("Graphe global", comment=f'Graphe global', format="png")
    colors = {"i": "white", "m": "#C9E0FF", "e": "#BCC9FF", "o": "#BBAAFF",
              "p": "#FF969F", "c": "#FFD1D9"}
    for t in "glntz": 
        colors[t] = "white"
    """for c in colors:
        colors[c] = "red" """
    dot.attr(rankdir='LR')
    dot.attr(size=size)
    # Ajout de tous les noeuds pour les oeuvres
    liste_edges = []
    list_nodes_in_links = set()
    for paire in list_links:
        for l in paire:
            list_nodes_in_links.add(l)
    for e in dict_entities:
        if e in list_nodes_in_links:
            dot.node(e, f"{dict_entities[e].label} [{dict_entities[e].type}]", 
                     color=colors[dict_entities[e].type], style="filled",
                     fillcolor=colors[dict_entities[e].type])
    for e in dict_entities:
        if dict_entities[e].type == "o":
            for resp in dict_entities[e].respIds:
                if resp not in list_nodes_in_links:
                    list_nodes_in_links.add(resp)
                    dot.node(e, f"{dict_entities[resp].label} [{dict_entities[resp].type}]", 
                     color=colors[dict_entities[resp].type], style="filled",
                     fillcolor=colors[dict_entities[resp].type])
                dot.edge(resp, e, label="Auteur")
            for i in dict_entities[e].subjects:
                if i not in list_nodes_in_links:
                    list_nodes_in_links.add(i)
                    dot.node(e, f"{dict_entities[i].label} [{dict_entities[i].type}]", 
                     color=colors[dict_entities[i].type], style="filled",
                     fillcolor=colors[dict_entities[i].type])
                dot.edge(i, e, label="Sujet")
            for gf in dict_entities[e].genreforme:
                if gf not in list_nodes_in_links:
                    list_nodes_in_links.add(gf)
                    dot.node(e, f"{dict_entities[gf].label} [{dict_entities[gf].type}]", 
                     color=colors[dict_entities[gf].type], style="filled",
                     fillcolor=colors[dict_entities[gf].type])
                dot.edge(gf, e, label="genre")
        if dict_entities[e].type == "e":
            for o in dict_entities[e].toOeuvres:
                dot.edge(e, o, label="Expression de")
        elif dict_entities[e].type == "m":
            for expr in dict_entities[e].toExpressions:
                dot.edge(e, expr, label="Manifestation de")
        elif dict_entities[e].type == "i":
            for m in dict_entities[e].toManifs:
                dot.edge(e, m, label="Item de")
    return dot
    # dot.edges(liste_edges)


In [ ]:
global_graph = generate_global_graph(dict_entities, liste_liens, "100")
global_graph.render("Graphe global")

# Coquilles rencontrées

https://semestriel.framapad.org/p/bugs_unimarclrm

In [ ]:
with open("stats_zones.txt", "w", encoding="utf-8") as file:
    for entity in dict_entities:
        for zone in dict_entities[entity].stats_zones:
            line = [entity, dict_entities[entity].type, zone, str(dict_entities[entity].stats_zones[zone])]
            file.write("\t".join(line) + "\n")

Expressions sans oeuvre

In [ ]:
for expr in expressions:
    if len(expr.toOeuvres) == 0:
        print(expr.id)#

In [ ]:
# Longueur des guides.

for el in dict_entities:
    guide = sru.record2fieldvalue(dict_entities[el].xml, "000")
    print(f"{dict_entities[el].type}¤{el}¤{guide}")

In [ ]:
for rec in dict_entities:
    for resp in dict_entities[rec].respIds:
        if resp not in dict_entities:
            print(rec, dict_entities[rec].type, resp)

In [ ]:
for el in autres_entites:
    print(el.id)

# Pseudo-moteur de recherche 

L'objectif est de proposer une boîte de saisie pour chercher un ou plusieurs mots-clés, et évaluer les résultats d'affichage possible


In [ ]:
def clean_str(string):
    string = unidecode(string.lower())
    punct = "!:,;.?/%$\"'"
    for char in punct:
        string = string.replace("char", " ")
    string = " ".join([el for el in string.split(" ") if el])
    return string

def search(keywords, oeuvres, dict_entities, index="all"):
    keywords = clean_str(keywords).split(" ")
    results_entities = {}
    for o in oeuvres:
        if index == "all":
            for k in keywords:
                if k in o.global_index:
                    results_entities[o.id] = o
    return results_entities

def short_display(results_entities):
    i = 1
    for e in results_entities:
        print("")
        print(f"{str(i)}.",  results_entities[e].label)
        print("")
        if i < len(results_entities) :
            print("-"*20)
        i += 1
    
def full_display(results_entities, kw_search):
    i = 1
    for e in results_entities:
        print("")
        print(f"{str(i)}.",  results_entities[e].detailed)
        print("")
        if i < len(results_entities) :
            print("-"*20)
        i += 1
    
    
def display_results(results_entities, kw_search):
    
    print("\nRappel de la recherche : ", kw_search)
    print("Nombre de résultats :", len(results_entities))
    short_display(results_entities)
    print("\n", "*"*20, "\n*    Résultats détaillés   *\n", "*"*20, "\n")
    full_display(results_entities, kw_search)

### Formulaire de recherche simple

In [ ]:
kw_search = input("Chercher dans la base : ")
results = search(kw_search, oeuvres, dict_entities, "all")
display_results(results, kw_search)

### Recherche avec pages HTML

La recherche génére une page de résultats dans [results/short_results.html](results/short_results.html), laquelle contient des liens vers des notices détaillées

On peut ensuite supprimer toutes les pages générées en utilisant la fonction `delete_html_results()`

Voir le noteboook [Jeu de données Unimarc(entités) - affichage des résultats](Jeu%20de%20donn%C3%A9es%20Unimarc(entit%C3%A9s)%20-%20affichage%20des%20r%C3%A9sultats.ipynb)

In [ ]:
from generate_results import display_html_results, delete_html_results

In [ ]:
kw_search = input("Chercher dans la base : ")
results = search(kw_search, dict_entities, "all")
print("Nombre de résultats :", len(results))
display_html_results(results, dict_entities, kw_search)

**[Afficher les résultats](results/short_results.html)**

#### Supprimer les pages de résultats

In [ ]:
# delete_html_results()

# Corrections à faire

Notice d'oeuvre UMLRM0163 : 2e zone 640 erronée :
```640 6  $0 Date de première sortie en salle $a US $d New ```

Notice de manif : lien 716$3 FRBNF13876673 --> FRBNF138766730 (notice de marque)

Coquilles sur les identifiants en général